In [1]:
"""
IS2_S2_cryo2ice_search.ipynb
Alek Petty, adapted from an earlier M. Bagnardi script.


https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-e84-aws.html

https://github.com/microsoft/PlanetaryComputerExamples/blob/main/tutorials/cloudless-mosaic-sentinel2.ipynb

GEE


Deescription:
- This notebook searches for semi-coincident ICESat-2 and Sentinel-2 data.
- The code finds Sentinel-2 images with ICESat-2 data falling within the image's footprint.
- The code was updated to read in ATl07 data from the new cloud storage using earthdata, see relevant tutorials:
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/ATL06-direct-access_rendered.ipynb
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/nsidc_daac_uwg_cloud_access_tutorial_rendered.ipynb (No longer hosted!!)
- Include new filtering to only look for images where we have cryo2ice (CryoSat-2 coincident) data based on external file. 
 
To do:
- Integrate the GEE or other cloud-based S-2 catalog. What data is on those comapred to the sentinelhub?
- Check all beams iunstead of just the middle beam.
- Add in other imagery/satellite sensors.

Notes:
- Another way of doing this is to use the RGT (reference ground track) data from ICESat-2.
    This could help if you want to look at future data I guess, but downside is it doesn't say where the data actually is on yhe ground, 
    just theoretical tracks.
- Uses the middle strong beam but could be easily adapted to check across all beams.=
- Tested with the adapted 'geo' conda env on cryocloud if you want to convert into a python script.

Output:

Update history:
 - Current notebook created in Apr 2024.

"""


"\nIS2_S2_cryo2ice_search.ipynb\nAlek Petty, adapted from an earlier M. Bagnardi script.\n\n\nGEE\n\n\nDeescription:\n- This notebook searches for semi-coincident ICESat-2 and Sentinel-2 data.\n- The code finds Sentinel-2 images with ICESat-2 data falling within the image's footprint.\n- The code was updated to read in ATl07 data from the new cloud storage using earthdata, see relevant tutorials:\n     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/ATL06-direct-access_rendered.ipynb\n     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/nsidc_daac_uwg_cloud_access_tutorial_rendered.ipynb (No longer hosted!!)\n- Include new filtering to only look for images where we have cryo2ice (CryoSat-2 coincident) data based on external file. \n \nTo do:\n- Integrate the GEE or other cloud-based S-2 catalog. What data is on those comapred to the sentinelhub?\n- Check all beams iunstead of just the middle beam.\n- Add in

In [2]:
# Uncomment these out when running for the first time on cryocloud
#%pip install sentinelsat
#%pip install earthdata
#%pip install astropy

In [3]:
#from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import datetime as dt
import h5py
import pandas as pd
# Need to do this for geopandas for some reason
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import glob
from shapely.geometry import Point, shape
import matplotlib.pyplot as plt
import os
from astropy.time import Time
import numpy as np
from PIL import Image
import io
import requests
import time
import earthaccess
import xarray as xr
from datetime import datetime
from datetime import timedelta

from pystac_client import Client

In [5]:
# ICESat-2/earthaccess stuff
auth = earthaccess.login()

#granules_cloud = earthaccess.search_data(
#    short_name = 'ATL07',
#    version = '006',
#    cloud_hosted = True,
#    bounding_box = (-180,60,180,90),
#    temporal = ('2020-10-01','2020-10-30'),
#    granule_name='ATL07-01_20201015012229_03190901_006_01.h5',
#)

# Open files (not found a way of not needing to do this..)
#atl07_files = earthaccess.open(granules_cloud)

Enter your Earthdata Login username:  akpetty
Enter your Earthdata password:  ········


In [6]:
#import icepyx as ipx
## Define variables for IceSat-2 (IS2) dataset query and granule retrieval.
#IS2_short_name = 'ATL07'
#IS2_version = '006'
#IS2_date_range = ['2020-10-01','2020-10-30']
#IS2_spatial_extent = [-180,60,180,90]

#IS2_query = ipx.Query(IS2_short_name, IS2_spatial_extent, IS2_date_range)
#IS2_file_names_ipyx = IS2_query.avail_granules(ids=True,cloud=True)[1]
#len(IS2_file_names)

In [7]:
c2ice_df = pd.read_csv('./Data/Cryo2Ice_info_array.csv')
c2ice_df['is2_start_time_dt'] = c2ice_df['start_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)

c2ice_df['is2_end_time_dt'] = c2ice_df['end_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)



In [8]:
# Search across the icepyx list
# take off the last version string (01/02 etc) as these change
#matched_data_granules=[]
#for file in c2ice_df["atl07filename"].tolist():
#    [matched_data_granules.append(string) for string in IS2_file_names_ipyx if string[-42:-6] in file]
#matched_data_granules

In [9]:
# Search across the icepyx list
# take off the last version string (01/02 etc) as these change
#matched_data_granules_ea=[]
#for file in c2ice_df["atl07filename"].tolist():
#    [matched_data_granules_ea.append(string) for string in atl07_files if string[-42:-6] in file]
#matched_data_granules_ea

In [10]:
    # Check spacecraft orientation and assign string beam IDs
    #ATL = h5py.File(atl07_file, 'r')
    
    #reader = ipx.Read(atl07_file)
    #reader.vars.append(var_list=['data_start_utc', 
    #                             'data_end_utc', 'longitude', 'longitude', 
    #                             'atlas_sdp_gps_epoch', 'longitude', 'delta_time'])
    #ATL = reader.load()

In [11]:
#reader.vars.avail()
c2ice_files = c2ice_df["atl07filename"].tolist()

In [12]:
stac_url = "https://earth-search.aws.element84.com/v0"
client = Client.open(stac_url)

#find the matched S2 images for each IS2 file
S2_scenes=[]
#number of S2 images for each IS2 granule
number_of_scenes=[]
#feature collection for each IS2
feature_track=[] 
# initialize

ATL_mid_times=[]

x=0

# Loop over ICESat-2 granules
#Coaren by this number of data points to speed things up, can play with this more as desired
NPTS=10
# time difference in minutes
deltatime=180

for c2ice_file in c2ice_files:

    granules_cloud = earthaccess.search_data(
        short_name = 'ATL07',
        version = '006',
        cloud_hosted = True,
        bounding_box = (-180,60,180,90),
        #temporal = ('2020-10-01','2020-10-30'),
        granule_name=c2ice_file,
    )
    
    print(granules_cloud)
    
    # Open files (not found a way of not needing to do this..)
    atl07_file = earthaccess.open(granules_cloud)
    ATL = h5py.File(atl07_file[0], 'r')
    

    orientation = ATL['/orbit_info/sc_orient'][0]

    # Only use central strong beam locations
    if orientation == 0:
        beamID = 'gt2l'
    elif orientation == 1:
        beamID = 'gt2r'
    else:
        print('Spacecraft orientation not found.')

    # Extract data info from granule
    ATL_start_time = ATL['/ancillary_data/data_start_utc'][0]
    ATL_start = pd.to_datetime(ATL_start_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')
    ATL_end_time = ATL['/ancillary_data/data_end_utc'][0]
    ATL_end = pd.to_datetime(ATL_end_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')

    ATL_mid = ATL_start + (ATL_end-ATL_start)/2
    ATL_mid_times.append(ATL_mid)
    # Build dataframe with location of data
    #ATL_dF = pd.DataFrame({'Longitude': ATL[beamID + '/sea_ice_segments/longitude'][::NPTS],
    #                       'Latitude': ATL[beamID + '/sea_ice_segments/latitude'][::NPTS],
    #                       })
    #ATL_dF['coords'] = list(zip(ATL_dF['Longitude'], ATL_dF['Latitude']))
    #ATL_dF['coords'] = ATL_dF['coords'].apply(Point)

    atl07_coords = list(zip(ATL[beamID + '/sea_ice_segments/longitude'][::NPTS], ATL[beamID + '/sea_ice_segments/latitude'][::NPTS])) 
    
    # Time
    # Use astropy to convert from gps time to datetime
    GPS_epoch = ATL['ancillary_data/atlas_sdp_gps_epoch'][:]
    GPS_time = ATL[beamID + '/sea_ice_segments/delta_time'][::NPTS] + GPS_epoch
    ATL_tgps = Time(GPS_time, format='gps')
    ATL_utc = ATL_tgps.utc.iso
    #ATL_dF['UTC'] = ATL_utc

    #ATL_gfd = gpd.GeoDataFrame(ATL_dF, geometry='coords')
    #ATL_gfd = ATL_gfd.set_crs(4326, allow_override=True)

    #create a feature collection representing a line from a list of valid latitude and longitude coordinate points.
    feature_track_temp = ee.FeatureCollection(ee.Geometry.LineString(coords=atl07_coords, proj='EPSG:4326', geodesic=True))
    feature_track.append(feature_track_temp)
    roi = feature_track[x].geometry()
    
    # Query Sentinel-2 images using ATL07 file information

    search = client.search(
        collections=["sentinel-s2-l2a-cogs"], 
        datetime="DATE/DATE")

    items = list(search.get_items())
    print(f"Found {len(items)} items.")
    
    # For demonstration, select the first item
    stac_item = items[0]
    geometry = stac_item.geometry


    # Convert geometry to a shape
    geom_shape = 
    
    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame([stac_item.properties], geometry=[shape(geometry)], crs="EPSG:4326")



    #imagecollection=ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(roi)\
    #.filterDate(ATL_start- dt.timedelta(minutes=deltatime), ATL_end+ dt.timedelta(minutes=deltatime))\
    #.filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 100))\
    #.filter(ee.Filter.lt('NODATA_PIXEL_PERCENTAGE', 10))
    
    print('\nCoincident S2 images:', imagecollection.size().getInfo())
    if imagecollection.size().getInfo()>0:
        print('Coincident S2 image FOUND')
        
    S2_scenes.append(imagecollection)
    number_of_scenes.append(imagecollection.size().getInfo())  
    
    x+=1


Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': 48.92460331907133, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-10-15T01:44:07.383Z', 'EndingDateTime': '2020-10-15T01:50:33.617Z'}}
Size(MB): 181.3192491531372
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2020/10/15/ATL07-01_20201015012229_03190901_006_01.h5']]
Opening 1 granules, approx size: 0.18 GB
using provider: NSIDC_CPRD


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]


Coincident S2 images: 0
Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': -48.924131734392304, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2021-04-02T00:03:38.476Z', 'EndingDateTime': '2021-04-02T00:18:35.195Z'}}
Size(MB): 376.94387912750244
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2021/04/02/ATL07-01_20210401235002_01251101_006_02.h5']]
Opening 1 granules, approx size: 0.37 GB
using provider: NSIDC_CPRD


...

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]


Coincident S2 images: 0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
atl07_file[:]

In [ ]:
#find valid S2 index4
coincident_granules_idx = [i for i, value in enumerate(number_of_scenes) if value != 0]
coincident_granules_idx

In [ ]:
# In this section, we can check single coincident dataset on the map by assigning different values from non_zero_indices to test_x. 
# AP update: I dont need the time check here as it's in my function above!
granule_num=1
scene_number = coincident_granules_idx[granule_num]
S2_ids = [S2_scenes[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['id'] for i in range(0, number_of_scenes[scene_number])]  

#time_diff = 3600*4 # Target time difference between ATL07 and Sentinel-2 < 3 hours
#time_IS2 = datetime.strptime(IS2_file_name[test_x][-33:-19], "%Y%m%d%H%M%S")

Map = geemap.Map()

for i in range(0, number_of_scenes[scene_number]):
#for i in range(0, 1):
    time_start = S2_scenes[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['properties']['system:time_start']
    time_s2 = datetime(1970, 1, 1) + timedelta(seconds = time_start/1000)   
    
    img_name = os.path.basename(S2_ids[i])

    # Time difference between IS2 and S2 < defined time_diff
    #if abs(time_IS2-time_s2).seconds <= time_diff:

    print(f" {img_name}: Time difference = {np.int(abs(ATL_mid_times[granule_num]-time_s2).seconds/60)} minutes with ICESat-2 track")
    
    img = ee.Image(S2_ids[i]).select('B4','B3','B2')
    dim = img.getInfo()['bands'][0]['dimensions']
    if dim[0] > 10000 and dim[1] > 0:
        # # Download image into the local folder
        #geemap.download_ee_image(img, f"S2_{img_name}.tif", scale=50)
        Map.addLayer(img, {'min': 0, 'max': 10000}, img_name)
        #print(f" {img_name}: Time difference = {abs(time_IS2-time_s2).seconds/3600} hours with ICESat-2 track")
        #else:
        #    print(f"SKIP {img_name}: Not a full image")
   #else:
   #     print(f"SKIP {img_name}: Time difference > {time_diff/3600} hours with ICESat-2 track")
   #     Map.addLayer(img, {'min': 0, 'max': 10000}, img_name)
   #     print(f" {img_name}: Time difference = {abs(time_IS2-time_s2).seconds/3600} hours with ICESat-2 track")

Map.centerObject(img, zoom = 5)
Map.addLayer(feature_track[scene_number], {}, "IS2 track")


Map

In [ ]:
S2[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['properties']